Add Average CFB Season Weather Data to School Data

In [4]:
import pandas as pd
import requests
from datetime import datetime
from time import sleep

INPUT_CSV = '../../intermediate_files/schools_with_locations.csv'
OUTPUT_CSV = '../../intermediate_files/schools_with_weather.csv'
WEATHER_API_URL = 'https://archive-api.open-meteo.com/v1/archive'

In [5]:
def fetch_season_weather_data(latitude, longitude, start_date='2024-08-01', end_date='2024-12-31'):
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'start_date': start_date,
        'end_date': end_date,
        'hourly': [
            'temperature_2m',
            'relative_humidity_2m',
            'wind_speed_10m',
            'wind_direction_10m',
            'precipitation',
            'weather_code'
        ],
        'timezone': 'America/New_York'
    }

    response = requests.get(WEATHER_API_URL, params=params)
    response.raise_for_status()
    data = response.json()

    if 'hourly' in data and data['hourly']['time']:
        temperatures = data['hourly']['temperature_2m']
        humidities = data['hourly']['relative_humidity_2m']
        wind_speeds = data['hourly']['wind_speed_10m']
        wind_directions = data['hourly']['wind_direction_10m']
        precipitations = data['hourly']['precipitation']
        weather_codes = data['hourly']['weather_code']

        daytime_data = []
        for i, time_str in enumerate(data['hourly']['time']):
            hour = datetime.fromisoformat(time_str.replace('Z', '+00:00')).hour
            if 8 <= hour <= 20:  # Only do daytime
                daytime_data.append({
                    'temperature': temperatures[i],
                    'humidity': humidities[i],
                    'wind_speed': wind_speeds[i],
                    'wind_direction': wind_directions[i],
                    'precipitation': precipitations[i],
                    'weather_code': weather_codes[i]
                })

        if daytime_data:
            # Averages
            avg_weather = {
                'avg_temperature_2m': sum(d['temperature'] for d in daytime_data) / len(daytime_data),
                'avg_humidity_2m': sum(d['humidity'] for d in daytime_data) / len(daytime_data),
                'avg_wind_speed_10m': sum(d['wind_speed'] for d in daytime_data) / len(daytime_data),
                'avg_wind_direction_10m': sum(d['wind_direction'] for d in daytime_data) / len(daytime_data),
                'total_precipitation': sum(d['precipitation'] for d in daytime_data),
                'avg_weather_code': sum(d['weather_code'] for d in daytime_data) / len(daytime_data),
                'data_points': len(daytime_data)
            }
            return avg_weather

    return None

In [6]:
df = pd.read_csv(INPUT_CSV)

weather_columns = [
    'avg_temperature_2m',
    'avg_humidity_2m',
    'avg_wind_speed_10m',
    'avg_wind_direction_10m',
    'total_precipitation',
    'avg_weather_code',
    'weather_data_points'
]

for col in weather_columns:
    df[col] = None

successful_requests = 0
failed_requests = 0

for idx, row in df.iterrows():
    school = row['cfb_school']
    lat = row['latitude']
    lon = row['longitude']

    if pd.isna(lat) or pd.isna(lon):
        print("Missing coordinates for", school)
        failed_requests += 1
        continue

    print("Fetching weather for", school)

    weather_data = fetch_season_weather_data(lat, lon)

    if weather_data:
        for col, value in weather_data.items():
            if col == 'data_points':
                df.at[idx, 'weather_data_points'] = value
            else:
                df.at[idx, col] = value
        successful_requests += 1
    else:
        failed_requests += 1

    sleep(5)
    print("Fetching weather for school", idx+1, "/", len(df))


    # Save every 10     
    if (idx + 1) % 10 == 0:
        print("RESULTS: ", successful_requests, failed_requests)
        df.to_csv(OUTPUT_CSV, index=False)

df.to_csv(OUTPUT_CSV, index=False)

Fetching weather for Abilene Christian
Fetching weather for school 1 / 252
Fetching weather for Air Force
Fetching weather for school 2 / 252
Fetching weather for Akron
Fetching weather for school 3 / 252
Fetching weather for Alabama
Fetching weather for school 4 / 252
Fetching weather for Alabama A&M
Fetching weather for school 5 / 252
Fetching weather for Alabama State
Fetching weather for school 6 / 252
Fetching weather for Albany
Fetching weather for school 7 / 252
Fetching weather for Alcorn State
Fetching weather for school 8 / 252
Fetching weather for Appalachian State
Fetching weather for school 9 / 252
Fetching weather for Arizona
Fetching weather for school 10 / 252
RESULTS:  10 0
Fetching weather for Arizona State
Fetching weather for school 11 / 252
Fetching weather for Arkansas
Fetching weather for school 12 / 252
Fetching weather for Arkansas State
Fetching weather for school 13 / 252
Fetching weather for Arkansas-Pine Bluff
Fetching weather for school 14 / 252
Fetching w